# Chapter 2 RDD(Resilient Distributed Datasets)
### 내부작동원리
----------------
스파크의 가장 큰 장점 - 병렬로 동작하는 RDD / 
각 트랜스포메이션은 속도를 비약적으로 향상시키기 위해 실행 / 
모든 트랜스포메이션은 데이터셋에 대한 액션이 호출됐을 때 실행 

### RDD 생성하기
-------------------
RDD를 생성하는 방법은 두가지이다.
1. 컬렉션에 대해 parallelize()함수를 수행

2. 파일 불러오기

In [3]:
data = sc.parallelize(
    [('Amber', 22), ('Alfred', 23), ('Skye',4), ('Albert', 12), 
     ('Amber', 9)])

보통 한 클러스터에서 2~4개 정도의 파티션으로 데이터 셋을 나누는 것이 좋다.
sc.textFile(A, n)에서 n은 데이터셋이 나눠진 파티션 개수를 의미

In [6]:
data_from_file = sc.textFile('/home/scio/jupyter_notebook/data/VS14MORT.txt.gz',4)

**Schema**

RDD는 데이터프레임과 달리 스키마리스 구조. 데이터셋에 대해 .collect() 함수를 수행하면 파이썬에서도 객체 내의 데이터에 접근할 수 있다

.collect() 함수는 RDD의 모든 엘리먼트를 드라이버에 리턴, 드라이버에서 엘리먼트들은 리스트로 나열

In [7]:
data_heterogenous = sc.parallelize([('Ferrari', 'fast'), {'Porsche': 100000}, ['Spain','visited', 4504]]).collect()
data_heterogenous

[('Ferrari', 'fast'), {'Porsche': 100000}, ['Spain', 'visited', 4504]]

In [8]:
data_heterogenous[1]['Porsche']

100000

**파일로부터 데이터 읽기**



텍스트 파일을 읽을때 파일의 각 행이 RDD의 한 엘리먼트를 이룬다.

In [9]:
data_from_file.take(1)

['                   1                                          2101  M1087 432311  4M4                2014U7CN                                    I64 238 070   24 0111I64                                                                                                                                                                           01 I64                                                                                                  01  11                                 100 601']

**Lambda**


주의: 일반적인 파이썬 함수 선언시 스파크가 파이썬 인터프리터와 JVM을 지속적으로 스위치 하기에 느려질 수 있다. 가능하면 반드시 스파크 내장 함수를 사용하자

In [11]:
def extractInformation(row):
    import re
    import numpy as np

    selected_indices = [
         2,4,5,6,7,9,10,11,12,13,14,15,16,17,18,
         19,21,22,23,24,25,27,28,29,30,32,33,34,
         36,37,38,39,40,41,42,43,44,45,46,47,48,
         49,50,51,52,53,54,55,56,58,60,61,62,63,
         64,65,66,67,68,69,70,71,72,73,74,75,76,
         77,78,79,81,82,83,84,85,87,89
    ]

    '''
        Input record schema
        schema: n-m (o) -- xxx
            n - position from
            m - position to
            o - number of characters
            xxx - description
        1. 1-19 (19) -- reserved positions
        2. 20 (1) -- resident status
        3. 21-60 (40) -- reserved positions
        4. 61-62 (2) -- education code (1989 revision)
        5. 63 (1) -- education code (2003 revision)
        6. 64 (1) -- education reporting flag
        7. 65-66 (2) -- month of death
        8. 67-68 (2) -- reserved positions
        9. 69 (1) -- sex
        10. 70 (1) -- age: 1-years, 2-months, 4-days, 5-hours, 6-minutes, 9-not stated
        11. 71-73 (3) -- number of units (years, months etc)
        12. 74 (1) -- age substitution flag (if the age reported in positions 70-74 is calculated using dates of birth and death)
        13. 75-76 (2) -- age recoded into 52 categories
        14. 77-78 (2) -- age recoded into 27 categories
        15. 79-80 (2) -- age recoded into 12 categories
        16. 81-82 (2) -- infant age recoded into 22 categories
        17. 83 (1) -- place of death
        18. 84 (1) -- marital status
        19. 85 (1) -- day of the week of death
        20. 86-101 (16) -- reserved positions
        21. 102-105 (4) -- current year
        22. 106 (1) -- injury at work
        23. 107 (1) -- manner of death
        24. 108 (1) -- manner of disposition
        25. 109 (1) -- autopsy
        26. 110-143 (34) -- reserved positions
        27. 144 (1) -- activity code
        28. 145 (1) -- place of injury
        29. 146-149 (4) -- ICD code
        30. 150-152 (3) -- 358 cause recode
        31. 153 (1) -- reserved position
        32. 154-156 (3) -- 113 cause recode
        33. 157-159 (3) -- 130 infant cause recode
        34. 160-161 (2) -- 39 cause recode
        35. 162 (1) -- reserved position
        36. 163-164 (2) -- number of entity-axis conditions
        37-56. 165-304 (140) -- list of up to 20 conditions
        57. 305-340 (36) -- reserved positions
        58. 341-342 (2) -- number of record axis conditions
        59. 343 (1) -- reserved position
        60-79. 344-443 (100) -- record axis conditions
        80. 444 (1) -- reserve position
        81. 445-446 (2) -- race
        82. 447 (1) -- bridged race flag
        83. 448 (1) -- race imputation flag
        84. 449 (1) -- race recode (3 categories)
        85. 450 (1) -- race recode (5 categories)
        86. 461-483 (33) -- reserved positions
        87. 484-486 (3) -- Hispanic origin
        88. 487 (1) -- reserved
        89. 488 (1) -- Hispanic origin/race recode
     '''

    record_split = re\
        .compile(
            r'([\s]{19})([0-9]{1})([\s]{40})([0-9\s]{2})([0-9\s]{1})([0-9]{1})([0-9]{2})' + 
            r'([\s]{2})([FM]{1})([0-9]{1})([0-9]{3})([0-9\s]{1})([0-9]{2})([0-9]{2})' + 
            r'([0-9]{2})([0-9\s]{2})([0-9]{1})([SMWDU]{1})([0-9]{1})([\s]{16})([0-9]{4})' +
            r'([YNU]{1})([0-9\s]{1})([BCOU]{1})([YNU]{1})([\s]{34})([0-9\s]{1})([0-9\s]{1})' +
            r'([A-Z0-9\s]{4})([0-9]{3})([\s]{1})([0-9\s]{3})([0-9\s]{3})([0-9\s]{2})([\s]{1})' + 
            r'([0-9\s]{2})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([\s]{36})([A-Z0-9\s]{2})([\s]{1})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([\s]{1})([0-9\s]{2})([0-9\s]{1})' + 
            r'([0-9\s]{1})([0-9\s]{1})([0-9\s]{1})([\s]{33})([0-9\s]{3})([0-9\s]{1})([0-9\s]{1})')
    try:
        rs = np.array(record_split.split(row))[selected_indices]
    except:
        rs = np.array(['-99'] * len(selected_indices))
    return rs
#     return record_split.split(row)

데이터셋을 쪼개고 변형하기 위해 extractinformation() 사용. map() 함수에 오로지 함수 시그니처만 전달(이 함수는 각 파티션에서 RDD 내의 한 데이터만 extractinformation() 함수에 전달)

In [12]:
data_from_file_conv = data_from_file.map(extractInformation)
data_from_file_conv.map(lambda row: row).take(1)

[array(['1', '  ', '2', '1', '01', 'M', '1', '087', ' ', '43', '23', '11',
        '  ', '4', 'M', '4', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'I64 ',
        '238', '070', '   ', '24', '01', '11I64  ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '01',
        'I64  ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], dtype='<U40')]

### 전역 범위 vs 지역 범위
----------------

스파크는 로컬 모드, 클러스터 모드로 동작한다. 스파크가 로컬모드로 동작할 때는 파이썬을 실행시키는 것과 다르지 않을 수도 있다.
참고: https://spark.apache.org/docs/latest/rdd-programming-guide.html#local-vs-cluster-modes 

### Transformations
------------------
필터링, 조인, 데이터셋 내의 값들에 대한 트랜스코딩등을 포함한 데이터셋의 형태를 만드는 것.

**.map()**

가장 많이 쓰이는 함수. 이 함수는 RDD의 각 엘리먼트에 적용된다. data_from_file_conv 데이터셋에서 이 함수를 각각의 행에 대한 트렌스포메이션으로 볼 수 있다. 리스트 형태로 리턴

In [26]:
data_2014 = data_from_file_conv.map(lambda row: int(row[16])) #사망날짜를 숫자값으로 변형
data_2014_2 = data_from_file_conv.map(lambda row: (row[16], row[5])) #사망날짜와 성별
print("data_2014.take(10)\n",data_2014.take(10))
print("\ndata_2014_2.take(10)\n",data_2014_2.take(10))

data_2014.take(10)
 [2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, -99]

data_2014_2.take(10)
 [('2014', 'M'), ('2014', 'M'), ('2014', 'F'), ('2014', 'M'), ('2014', 'M'), ('2014', 'F'), ('2014', 'M'), ('2014', 'M'), ('2014', 'F'), ('-99', '-99')]


**.flatMap()**

.map() 함수와 비슷하게 동작. 그러나 평면화된 결과를 리턴

In [33]:
data_2014_flat = data_from_file_conv.flatMap(lambda row: (int(row[16]), row[5]))
data_2014_flat.take(10)

[2014, 'M', 2014, 'M', 2014, 'F', 2014, 'M', 2014, 'M']

**.filter()**

특정 조건에 맞는 엘리먼트 선택 가능. 2014년에 사고사 인원을 카운트 해보자

In [42]:
data_filtered = data_from_file_conv.filter(
    lambda row: row[16]=='2014' and row[21] =='0')
data_filtered.count()

22